# Sparseloop Tutorial - 02 - Matrix Multiply

This notebook contains a series of examples of a **matrix multiply** computation. The **fibertree** emulator is used to illustrate the impact of a set of optimizations to exploit sparsity. The basic computation is represented by the Einsum:

$$ Z_{m,n} = A_{m,k} \times B_{k,n} $$

Note that while the output is nominally a sparse rank-2 tensor, in this notebook the output is assumed to be in an uncompressed format and is directly referenced by **coordinate** since **position** == **coordinate** in an uncompressed format.

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run ./prelude.py --style=tree --animation=movie

## Configure two rank-2 input tensors

The following cell sets up the control sliders to specify the attributes of the `A` and `B` input tensors. Those attributes include their **shape**, which specifies the allowable range of **coordinates** of elements of the tensor and their **density**.

The rank names use the following convention:

- M - uncontracted dimension in input `A`
- N - uncontracted dimension in input `B`
- K - contracted dimension shared by `A` and `B`



In [ ]:
#
# Set default problem instance attributes (i.e., the shape of the tensors)
#
K = 8
M = 6
N = 6

#
# Create controls to configure the `A` and `B` tensors
#
tm2 = TensorMaker("sparseloop-matrix-multiply", autoload=True)

tm2.addTensor("A", rank_ids=["M", "K"], shape=[M, K], density=0.4, color="blue")
tm2.addTensor("B", rank_ids=["K", "N"], shape=[K, N], density=0.5, color="green")

tm2.displayControls()


## Create and display the input tensors

Create the `A` and `B` tensors based on the current settings of the configuration sliders above and display the resulting tensors. These tensors are represented in the **fibertree** tensor abstraction, where for sparse fibers only the **elements** (**coordinate**/**payload** tuples) in a fiber with **non-empty** (non-zero) payloads need be shown.

In [ ]:
A_MK = tm2.makeTensor("A")
B_KN = tm2.makeTensor("B")

displayTensor(A_MK)
displayTensor(B_KN)

# Traverse a rank-2 tensor

As a simple illustration of the traversal of a rank-2 tensor, the following code shows how that can be accomplished with two nested `for` loops where the **payload** of the traversal of the top rank is a reference to a **fiber** in the next rank.

This natural traversal order where the bottom rank is traversed fastest (i.e., in a depth first fashion) is referred to as a **concordant** traversal and tend to be the most efficient. Other, less efficient, traversal orders are referred to as **discordant**.


In [ ]:
#
# Create a rank-2 tensor
#
A_MK = tm2.makeTensor("A")

#
# Get root of tensor
#
a_m = A_MK.getRoot()

#
# Animation bookkeeeping
#
canvas = createCanvas(A_MK)

cycle = 0

#
# Traverse non-empty elements of top rank
#
for m, a_k in a_m:
    #
    # Traverse non-empty element of bottom rank
    #
    for k, a_val in a_k:
        
        print(f"{a_val} ", end='')
            
        #
        # Animation bookkeeping
        #
        canvas.addActivity((m,k), spacetime=(0,cycle))
        cycle += 1

displayCanvas(canvas)

# Matrix multiply

The following code illustrates the traversal and accesses associated with the computation of untiled matrix multiply as expressed by the following Einsum:

$$ Z_{m,n} = A_{m,k} \times B_{k,n} $$

In [ ]:
#
# Create the input/output tensors
#
K = getShape(tm2, "K")
M = getShape(tm2, "M")
N = getShape(tm2, "N")

A_MK = tm2.makeTensor("A")
B_KN = tm2.makeTensor("B")

Z_MN = Tensor(name="Z", rank_ids=["M", "N"], shape=[M, N])

uncompressTensor(Z_MN)

#
# Display the tensors
#
print("Problem instance:")
print(f"M: {M}")
print(f"K: {K}")
print(f"N: {N}")
print("")

print("Input A")
displayTensor(A_MK)
print("Input B")
displayTensor(B_KN)
print("Output Z (initial)")
displayTensor(Z_MN)

#
# Get the root fibers of each tensor
#
a_m = A_MK.getRoot()
b_k = B_KN.getRoot()
z_m = Z_MN.getRoot()

#
# Animation bookkeeping
#
canvas = createCanvas(A_MK, B_KN, Z_MN)

cycle = 0

#
# Traverse non-empty elements of top rank of `A`
#
for m, a_k in a_m:
    #
    # Traverse the K rank of `A`
    #
    for k, a_val in a_k:
        #
        # Obtain the matching fiber in `B`
        #
        b_n = b_k.getPayload(k)
        
        #
        # Traverse the bottom rank of `B`
        #
        for n, b_val in b_n:
            #
            # Do the reduction
            #
            z_m[m][n] += a_val * b_val
            
            #
            # Animation bookkeeping
            #
            canvas.addActivity((m,k), (k,n), (m, n), spacetime=(0,cycle))
            cycle += 1
            
#
# Display results
#
print("Output tensor Z (final)")
displayTensor(Z_MN)

displayCanvas(canvas)

# Tiled Matrix Multiply

$$ Z_{n1,m,n0} = A_{m,k} \times B_{n1,k,n0}$$

## Splitting of B

First we look at the sequence for splitting the `B` tensor along the N rank.

Notes:

- The coordinates in the N0 rank are the original N coordinates
- We are assuming that this is done offline... Online tiling is beyond the scope of this notebook.

In [ ]:
#
# Create tensors
#

N0 = 3

B_KN = tm2.makeTensor("B")

print("Original B tensor")
displayTensor(B_KN)

#
# Split the `N` rank of `B` into a series of fibers with shape `N0`
# then rename the coordinates of the N1 rank to match the coordinates we want in the output (hacky)
#
B_N1N0K = B_KN.splitUniform(N0, depth=1)
B_N1N0K = B_N1N0K.updateCoords(lambda n, c, p: n, depth=1)

print("B tensor with N rank split into fibers with a shape={N0}")
displayTensor(B_N1N0K)

#
# Swizzle the ranks of B so that we can traverse it in a **concordant** fashion
#
B_N1KN0 = B_N1N0K.swapRanks()

print("B tensor split and swizzed for concordant traversal")
displayTensor(B_N1KN0)

## Tiled Matrix Multiply

In [ ]:
#
# Create the input/output tensors
#
K = getShape(tm2, "K")
M = getShape(tm2, "M")
N = getShape(tm2, "N")

N1 = 2
N0 = (N+1)//2

#
# Create tensors
#

A_MK = tm2.makeTensor("A")
B_KN = tm2.makeTensor("B")


B_N1KN0 = B_KN.splitUniform(N0, depth=1).swapRanks()
B_N1KN0 = B_N1KN0.updateCoords(lambda n, c, p: n)

Z_MN = Tensor(name="Z",
              rank_ids=["M", "N"],
                 shape=[M, N])
uncompressTensor(Z_MN)

Z_N1MN0 = Z_MN.splitUniform(N0, depth=1).swapRanks()
Z_N1MN0 = Z_N1MN0.updateCoords(lambda n, c, p: n)
Z_N1MN0.setMutable(True)

#
# Display Tensors
#
print("Problem instance:")
print(f"K:  {K}")
print(f"M:  {M}")
print(f"N:  {N}")
print(f"N1: {N1}")
print(f"N0: {N0}")

displayTensor(A_MK)
displayTensor(B_N1KN0)
displayTensor(Z_N1MN0)

#
# Get the root fibers of each tensor
#
a_m = A_MK.getRoot()
b_n1 = B_N1KN0.getRoot()
z_n1 = Z_N1MN0.getRoot()

#
# Animation bookkeeping
#
canvas = createCanvas(A_MK, B_N1KN0, Z_N1MN0)

cycle = 0

#
# Traverse non-empty elements of `N1` (top) rank of `B`
#
for n1, b_k in b_n1:
    #
    # Traverse non-empty elements of `M` (top) rank of A
    #
    for m, a_k in a_m:
        #
        # Traverse the K (bottom) rank of `A`
        #
        for k, a_val in a_k:
            #
            # Obtain the matching fiber in `B`
            #
            b_n0 = b_k.getPayload(k)
        
            #
            # Traverse the `N0` (bottom) rank of `B`
            #
            for n0, b_val in b_n0:
                #
                # Do the reduction
                #
                # Note hack to get right position in `N0` rank of `Z`
                #
                z_n1[n1][m][n0%N0] += a_val * b_val
            
                #
                # Animation bookkeeping
                #
                canvas.addActivity([(m,k)], [(n1,k,n0)], [(n1, m, n0)], 
                                   spacetime=(0,cycle))
                cycle += 1
            
displayTensor(Z_N1MN0)
displayCanvas(canvas)

## Testing area

For running alternative algorithms